In [1]:
#Import Necessary Libraries
from matplotlib.pyplot import *
import numpy as np
import matplotlib.pyplot as plt
import random
import math
from statistics import mode
from copy import deepcopy
import heapq
import pandas as pd

In [2]:
class LIF:
    def __init__(self, dt=0.125, u_rest=0, R=1, 
                 C=10, threshold=5):
        self.dt = dt
        self.u_rest = u_rest
        self.R = R
        self.C = C
        self.threshold = threshold
        self.u = []
        self.i_init = []
        self.timer = []
        self.is_spiked = False
        self.last_spike = -1
        
        self.spikes = []
        self.sike_count = 0
        self.curentU = 0
        
    def one_move(self, current_time, current_input):
        du = self.dt * (-1 * (self.curentU - self.u_rest) + 1e-3 * self.R * current_input) / (self.R * self.C)
        self.curentU += du
        current_time += du
        self.is_spiked = True
        if self.curentU > self.threshold:
            self.curentU =self.u_rest
            self.is_spiked = False
            self.last_spike = current_time 
            self.sike_count += 1
            self.spikes.append(current_time)
        self.u.append(self.curentU)
        
    def reset(self):
        self.u = []
        self.spikes = []
        self.sike_count = 0
        self.curentU = self.u_rest

In [3]:
 class SNN:
    def __init__(self, dimention, neurons, tau_t, tau_dope, r_score):
        self.dim = dimention
        self.neurons = neurons
        self.tau_t = tau_t
        self.tau_d = tau_dope
        self.r_score = r_score
        self.connectins = []
        self.tags = []
        self.dopamine = 0
        self.connect()
        
    def connect(self):
        for i in range(0, len(self.dim) - 1):
            self.connectins.append(np.ones((self.dim[i], self.dim[i+1])) * 8 + np.random.rand())
            self.tags.append(np.zeros((self.dim[i], self.dim[i+1])))
        
    def fit(self, inputs, winners, epoc_t,dt, dt_minus, dt_plus, a_minus, a_plus, ts_thresh, iterations, learn_time): 
        time = 0
        lt = learn_time
        
        for iteration in range(iterations):
            rand = np.random.randint(0, inputs.shape[0]) 
            epoch = 0
            
            while epoch < epoc_t:
                inp = inputs[rand]
                for layer in range(len(self.neurons)):
                    for i, neuron in enumerate(self.neurons[layer]):
                        neuron.one_move(time ,inp[i])
                    if layer != len(self.neurons) - 1:
                        inp = self.activities(layer, time + dt, dt, ts_thresh)@self.connectins[layer]
                flag = False
                if time > lt:
                    flag = True
                    lt += learn_time
                self.learn(winners, rand, time + dt, dt, dt_minus, dt_plus, a_minus, a_plus, learn_time, flag)
                epoch += dt
    
    def learn(self, winners_list, idx, time, dt, dt_minus, dt_plus, a_minus, a_plus, learn_time, flag):
        for layer in range(len(self.connectins)):
            for i in range(len(self.neurons[layer])):
                for j in range(len(self.neurons[layer + 1])):
                    preN = self.neurons[layer][i]
                    postN = self.neurons[layer][j]
                    stdp = 0
                    
                    if preN.sike_count != 0 and postN.sike_count != 0 and postN.last_spike == time:
                        dt = postN.last_spike - preN.last_spike
                        if dt > 0:
                            stdp += a_plus * math.exp(-(math.fabs(dt)/dt_plus))
                        elif dt < 0:
                            stdp += a_minus * math.exp(-(math.fabs(dt)/dt_minus))
                    
                    self.tags[layer][i, j] += dt * ((-self.tags[layer][i,j]/self.tau_t)+stdp)
                    self.connectins[layer][i,j] += dt * self.tags[layer][i, j] * self.dopamine
                    if self.connectins[layer][i,j] < 0:
                       self.connectins[layer][i,j] = 0
        reward = 0
        
        if flag:
            actions = np.zeros_like(self.neurons[-1])
            for i, neuron in enumerate(self.neurons[-1]):
                amount = 0
                for spike in neuron.spike:
                    if time - spike > learn_time:
                        amount += 1
                actions[i] = amount
            if np.argma(actions) == int(winners_list[idx]):
                largest_ints = heapq.nlargest(2, list(actions))
                if largest_ints[0] != 0 and(largest_ints[0] - largest_ints[1]) / largest_ints[0] > 0.1:
                    reward += self.r_score
                else:
                    reward -= self.r_score
            else:
                reward -= r_score
        self.dopamine += learn_time * ((-self.dopamine/self.tau_d +reward))
        
    def activities(self,layer, time, dt, ts_thresh): 
        activ_list = np.zeros(self.dim[layer])
        for i in range(self.dim[layer]):
            neuron = self.neurons[layer][i]
            s = 0
            activity = 0
            while self.t_course(s) > ts_thresh:
                if time - s in neuron.spikes:
                    activity += self.t_course(s)
                s += dt
            activ_list[i] = activity
        return activ_list
    
    
    def predict(self,inputs, time, dt, ts_thresh):
        resaults = np.zeros((len(inputs)))
        for i in range(len(inputs)):
            self.reset()
            t = 0
            while t < time:
                input= inputs[i]
                for layer in range(len(self.neurons)):
                    for k, neuron in enumerate(self.neurons[layer]):
                        neuron.one_move(t, input[k])
                        if layer != len(self.neurons) - 1:
                            inp = self.activities(layer, time + dt, dt, ts_thresh)@self.connectins[layer]
                t += dt
            print(f'{self.neurons[-1][0].sike_count}  {self.neurons[-1][1].sike_count} ')
            if self.neurons[-1][0].sike_count > self.neurons[-1][1].sike_count:
                resaults[i] = 0
            else:
                resaults[i] = 1

        return resaults

    def reset(self):
        for layer in self.neurons:
            for neuron in layer:
                neuron.reset()
    def t_course(self,s): 
        return 200 * (1/(8* math.sqrt(2 * math.pi))) * math.exp(-(s/8) ** 2)
    
        
        

In [4]:
dataset = pd.read_excel('./CN_3_DATASET.XLSX')

In [5]:
train_inputs = dataset.iloc[1:6,1:].values.T * 1000
train_outs = dataset.iloc[6, 1:].values
test_inputs = dataset.iloc[10:15,1:].values.T * 1000
test_outs = dataset.iloc[15, 1:].values

In [6]:
dim = (5, 2)

In [7]:
layer1 = []
for i in range(5):
    layer1.append(LIF(dt = 0.125, u_rest = random.randint(-3, 3), R=random.randint(1, 4), C=random.randint(8, 12), threshold=random.randint(4, 8)))

In [8]:
layer2 = []
for i in range(2):
    layer2.append(LIF(dt = 0.125, u_rest = random.randint(-3, 3), R=random.randint(1, 4), C=random.randint(8, 12), threshold=random.randint(5, 7)))

In [9]:
neurons = [layer1, layer2]


In [10]:
snn = SNN(dim, neurons, 15, 12, 7)

In [11]:
snn.fit(train_inputs, train_outs, 5, 0.125, 7, 9, -4, 7, 1, 100, 0.5)

In [12]:
a = snn.predict(test_inputs,100, 0.125, 1)

0  0 
0  0 
0  0 
0  0 
0  0 
0  0 
1  0 
0  0 
0  0 
0  0 


In [13]:

count = 0
for i, el in enumerate(a):
    if el == test_outs[i]:
        count +=1
print(count/len(a))
print(a)

0.7
[1. 1. 1. 1. 1. 1. 0. 1. 1. 1.]


In [14]:
a = snn.predict(train_inputs,100, 0.125, 1)

0  0 
0  0 
1  0 
0  0 
0  0 
0  0 
0  0 
0  0 
0  0 
0  0 


In [15]:
count = 0
for i, el in enumerate(a):
    if el == train_outs[i]:
        count +=1
print(count/len(a))
print(a)

0.5
[1. 1. 0. 1. 1. 1. 1. 1. 1. 1.]
